## RESPUESTAS

#### ¿Cuál será la previsión de transacciones para una tienda específica en los próximos 30 días?
#### (Predicción + gráfico de tendencia).

In [14]:
import pandas as pd
import numpy as np
import joblib
from datetime import timedelta
import matplotlib.pyplot as plt
import os

import os
import joblib

# Ruta dinámica al modelo
base_dir = os.path.abspath(os.path.join(os.getcwd(),".."))
modelo_path = os.path.join(base_dir,"models","random_forest_model.pkl")

cargado = joblib.load(modelo_path)
modelo = cargado['model']
encoders = cargado['encoders']

# 2️⃣ Simular input: tienda específica + 30 días futuros
store_nbr = 22  # Cambia esto por la tienda que desees
hoy = pd.to_datetime('2025-03-23')  # Supón que hoy es la fecha de predicción
futuras_fechas = pd.date_range(start=hoy, periods=30)
lugar = "Pichincha - Quito"
formato = "D"
cluster = 13
locale = None
# 3️⃣ Simular las features mínimas necesarias (ajusta según tus features reales)

df_pred = pd.DataFrame({
    'fecha': futuras_fechas,
    'lugar': lugar ,
    'idTienda': store_nbr,
    'formatoTienda': formato ,
    'cluster' : cluster,
    'precioGlobal': np.random.uniform(30, 100, size=30),  # Aquí pondrías precios reales o simulados
    'type': 0 ,  # Suponemos que no es festivo (puedes luego variar esto)
    'locale' : locale
})

# Extraer más info de la fecha si tu modelo lo necesita
df_pred['dia'] = df_pred['fecha'].dt.day
df_pred['mes'] = df_pred['fecha'].dt.month
df_pred['ano'] = df_pred['fecha'].dt.year

for col in df_pred.select_dtypes(include='object').columns:
    df_pred[col] = encoders[col].transform(df_pred[col].astype(str))

# 4️⃣ Predecir
features = df_pred.drop(columns=['fecha'])  # Ajusta si tu modelo no usa fecha
predicciones = modelo.predict(features)

media_mensual = predicciones.mean()
print(f"📦 Stock recomendado promedio para el mes: {media_mensual:.0f} unidades")






📦 Stock recomendado promedio para el mes: 836 unidades


In [15]:
tiendas = [1, 2, 3, 4, 5]  # Lista con IDs de tiendas reales de tu dataset
resultados = []

for tienda in tiendas:
    df_pred['idTienda'] = tienda
    pred = modelo.predict(df_pred.drop(columns=['fecha']))
    media = pred.mean()
    resultados.append((tienda, media))

peor_tienda = min(resultados, key=lambda x: x[1])
print(f"🚩 La tienda con menor volumen estimado es la tienda {peor_tienda[0]} con {peor_tienda[1]:.0f} unidades promedio.")


🚩 La tienda con menor volumen estimado es la tienda 1 con 1509 unidades promedio.


In [16]:
umbral_stock = 1500  # Defínelo según tu contexto
df_pred['predicciones'] = modelo.predict(df_pred.drop(columns=['fecha']))

riesgo = df_pred[df_pred['predicciones'] < umbral_stock]

if not riesgo.empty:
    print("⚠️ Días con riesgo de bajo stock:")
    print(riesgo[['fecha', 'predicciones']])
else:
    print("✅ No hay días críticos según el umbral definido.")


✅ No hay días críticos según el umbral definido.
